<a href="https://colab.research.google.com/github/sherrylittlefish/DLGAN/blob/main/pytorch_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>